# Movie Industry Analysis - Phase 2 Project
   **Author:** Angela Mukami  
   **Date:** December 2025

PROJECT OVERVIEW

Business Problem: Your company wants to create a new movie studio but lacks expertise in film production. 

You need to analyze what types of films perform best at the box office.

In [5]:
import sqlite3
import pandas as pd

# create connection to SQLite database
conn = sqlite3.connect('im.db')

# create cursor
cursor = conn.cursor()

In [8]:
# list all available tables
cursor.execute("""
SELECT name FROM sqlite_master 
WHERE type= 'table';
""")
cursor.fetchall()

[('movie_basics',),
 ('directors',),
 ('known_for',),
 ('movie_akas',),
 ('movie_ratings',),
 ('persons',),
 ('principals',),
 ('writers',)]

In [9]:
# Load just the first 5 rows to see the structure quickly
query1 = """SELECT * FROM movie_basics LIMIT 5;"""
df_movie_basics = pd.read_sql_query(query1, conn)
df_movie_basics.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [10]:
# Load first 5 rows from movie_ratings
query2 = """SELECT * FROM movie_ratings LIMIT 5;"""
df_movie_ratings = pd.read_sql_query(query2, conn)
df_movie_ratings.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


Both tables have movie_id, which means we can merge them together!

What we learned about movie_ratings:

movie_id - Matches the movie_id in movie_basics

averagerating - IMDB rating (0-10 scale, like 8.3, 8.9, 6.4)

numvotes - Number of people who voted (31, 559, 50352)

Something to notice: Movie with 50,352 votes is more reliable than one with only 31 votes!

In [14]:
# Load Box Office Mojo data
bom_df = pd.read_csv('bom.movie_gross.csv')

# Quick overview
print("📊 Box Office Mojo Dataset Overview:")
print("="*60)
print(f"Shape: {bom_df.shape}")
print(f"\nColumns: {bom_df.columns.tolist()}")
print(f"\nFirst 5 rows:")
bom_df.head()

📊 Box Office Mojo Dataset Overview:
Shape: (3387, 5)

Columns: ['title', 'studio', 'domestic_gross', 'foreign_gross', 'year']

First 5 rows:


,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [17]:
# Check year range
print("📅 Year Range:")
print(f"Earliest year: {bom_df['year'].min()}")
print(f"Latest year: {bom_df['year'].max()}")
print(f"\nMovies per year:")
print(bom_df['year'].value_counts().sort_index())

print("\n" + "="*60)

# Check for missing data
print("\n Missing Values:")
print(bom_df.isnull().sum())


📅 Year Range:
Earliest year: 2010
Latest year: 2018

Movies per year:
2010    328
2011    399
2012    400
2013    350
2014    395
2015    450
2016    436
2017    321
2018    308
Name: year, dtype: int64


 Missing Values:
title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64


Important decision: We'll focus on domestic_gross since it has much less missing data than foreign gross!


In [ ]:
# now we can go back and load IMDB data for 2010-2018 with filtering criteria
# 
# FILTERING RATIONALE:
# 1. Years 2010-2018: Match the Box Office Mojo dataset timeframe for accurate merging
# 2. numvotes >= 1000: Ensure rating reliability by filtering out movies with too few votes
#    - Movies with <1000 votes may have skewed/unreliable ratings
#    - Industry standard: minimum sample size for statistical significance
# 3. runtime_minutes IS NOT NULL: Need runtime data for our analysis
# 4. genres != 'NaN': Need genre information to answer business question about film types

query_imdb = """
SELECT 
    mb.movie_id,
    mb.primary_title,
    mb.start_year,
    mb.runtime_minutes,
    mb.genres,
    mr.averagerating,
    mr.numvotes
FROM movie_basics mb
INNER JOIN movie_ratings mr ON mb.movie_id = mr.movie_id
WHERE mb.start_year >= 2010
    AND mb.start_year <= 2018
    AND mb.runtime_minutes IS NOT NULL
    AND mb.genres != 'NaN'
    AND mr.numvotes >= 1000
ORDER BY mr.numvotes DESC;
"""

print("Loading IMDB data for 2010-2018...")
imdb_df = pd.read_sql_query(query_imdb, conn)

print(f"Loaded {len(imdb_df):,} movies from IMDB")
print(f"\nFirst 10 most popular movies:")
imdb_df.head(10)

Loading IMDB data for 2010-2018...
✅ Loaded 9,426 movies from IMDB

First 10 most popular movies:


,movie_id,primary_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt1375666,Inception,2010,148.0,"Action,Adventure,Sci-Fi",8.8,1841066
1,tt1345836,The Dark Knight Rises,2012,164.0,"Action,Thriller",8.4,1387769
2,tt0816692,Interstellar,2014,169.0,"Adventure,Drama,Sci-Fi",8.6,1299334
3,tt1853728,Django Unchained,2012,165.0,"Drama,Western",8.4,1211405
4,tt0848228,The Avengers,2012,143.0,"Action,Adventure,Sci-Fi",8.1,1183655
5,tt0993846,The Wolf of Wall Street,2013,180.0,"Biography,Crime,Drama",8.2,1035358
6,tt1130884,Shutter Island,2010,138.0,"Mystery,Thriller",8.1,1005960
7,tt2015381,Guardians of the Galaxy,2014,121.0,"Action,Adventure,Comedy",8.1,948394
8,tt1431045,Deadpool,2016,108.0,"Action,Adventure,Comedy",8.0,820847
9,tt1392170,The Hunger Games,2012,142.0,"Action,Adventure,Sci-Fi",7.2,795227
